# 🎬 Video Frame Extractor - Google Colab

Extract frames from videos and save directly to Google Drive!

## Features
- ⚡ Parallel processing for faster extraction
- 📥 Download videos from URLs (YouTube, direct links, etc.)
- 💾 Save directly to Google Drive
- 🖼️ High-quality PNG output

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Your Google Drive is now accessible at /content/drive/MyDrive/

## Step 2: Install Dependencies

**Important:** We need Node.js 18+ for this tool to work properly.

In [ ]:
# Install Node.js 18 (required for modern JavaScript features)
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!sudo apt-get install -y nodejs

# Install ffmpeg and yt-dlp
!apt-get install -qq ffmpeg
!pip install -q yt-dlp

# Verify installations
print("\n✅ Installed versions:")
!node --version
!npm --version
!ffmpeg -version 2>&1 | head -1
!yt-dlp --version

## Step 3: Clone and Setup Video Frame Extractor

In [ ]:
# Clone the repository (replace with your repo URL)
%cd /content
!rm -rf video-frame-extractor  # Remove if exists
!git clone https://github.com/YOUR_USERNAME/video-frame-extractor.git
%cd video-frame-extractor

# Install npm dependencies
!npm install

print("\n✅ Setup complete!")

## Step 4: Configure Paths

Set your input and output directories. Output will be saved directly to Google Drive!

In [ ]:
# Configuration
INPUT_VIDEOS = "/content/videos"  # Local folder for input videos
OUTPUT_FRAMES = "/content/drive/MyDrive/extracted_frames"  # Save to Google Drive!
DOWNLOADS_DIR = "/content/downloads"  # For downloaded videos

# Create directories
!mkdir -p {INPUT_VIDEOS}
!mkdir -p "{OUTPUT_FRAMES}"
!mkdir -p {DOWNLOADS_DIR}

print(f"✅ Input videos: {INPUT_VIDEOS}")
print(f"✅ Output frames: {OUTPUT_FRAMES} (Google Drive)")
print(f"✅ Downloads: {DOWNLOADS_DIR}")

## Option A: Extract from Uploaded Videos

Upload videos and extract frames.

In [ ]:
# Upload videos using Colab's file upload
from google.colab import files
import shutil
import os

print("Upload your video files:")
uploaded = files.upload()

# Move uploaded files to input directory
for filename in uploaded.keys():
    dest = os.path.join(INPUT_VIDEOS, filename)
    shutil.move(filename, dest)
    print(f"  ✅ Moved {filename} to {INPUT_VIDEOS}/")

In [ ]:
# Extract frames from uploaded videos → Save to Google Drive
%cd /content/video-frame-extractor
!node src/index.js \
    -i "{INPUT_VIDEOS}" \
    -o "{OUTPUT_FRAMES}" \
    -c 4 \
    --fps 1

print("\n✅ Frames saved to Google Drive!")

## Option B: Download from URLs and Extract

Download videos from URLs (YouTube, direct links, etc.) and extract frames.

In [ ]:
%%writefile /content/video-frame-extractor/links.txt
# Add your video URLs here (one per line)
# Lines starting with # are comments

# Example: YouTube video
# https://www.youtube.com/watch?v=VIDEO_ID

# Example: Direct video URL
# https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_1mb.mp4

In [ ]:
# Edit the links.txt file above with your URLs, then run this cell

# Download videos from URLs and extract frames → Save to Google Drive
%cd /content/video-frame-extractor
!node src/index.js \
    -u links.txt \
    -o "{OUTPUT_FRAMES}" \
    -d "{DOWNLOADS_DIR}" \
    -c 4 \
    --fps 1

print("\n✅ Frames saved to Google Drive!")

## Option C: Extract ALL Frames (Warning: Many Files!)

In [ ]:
# Extract ALL frames (no FPS limit - creates many files!)
%cd /content/video-frame-extractor
!node src/index.js \
    -i "{INPUT_VIDEOS}" \
    -o "{OUTPUT_FRAMES}" \
    -c 4 \
    -q 2

## Check Results

In [ ]:
import os

# List extracted frames in Google Drive
if os.path.exists(OUTPUT_FRAMES):
    folders = os.listdir(OUTPUT_FRAMES)
    print(f"📁 Found {len(folders)} video folder(s) in Google Drive:\n")
    for folder in folders:
        folder_path = os.path.join(OUTPUT_FRAMES, folder)
        if os.path.isdir(folder_path):
            files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
            print(f"  📂 {folder}: {len(files)} frames")
else:
    print("No frames extracted yet.")

In [ ]:
# Preview frames
from IPython.display import Image, display
import glob

frames = sorted(glob.glob(f"{OUTPUT_FRAMES}/**/*.png", recursive=True))
if frames:
    print(f"Found {len(frames)} total frames. Showing first 3:\n")
    for frame in frames[:3]:
        print(f"📷 {frame}")
        display(Image(filename=frame, width=400))
        print()
else:
    print("No frames found yet. Run one of the extraction options above.")

## Advanced Options

```bash
# Extract frames from 10s to 30s only
!node src/index.js -i INPUT -o OUTPUT --start 10 --end 30

# Extract at 0.5 FPS (1 frame every 2 seconds)
!node src/index.js -i INPUT -o OUTPUT --fps 0.5

# Best quality (slower, larger files)
!node src/index.js -i INPUT -o OUTPUT -q 0

# Fastest (lower quality, smaller files)
!node src/index.js -i INPUT -o OUTPUT -q 9

# Process 8 videos in parallel
!node src/index.js -i INPUT -o OUTPUT -c 8

# Custom output filename pattern
!node src/index.js -i INPUT -o OUTPUT -f "img_%08d.png"
```

## Cleanup (Optional)

In [ ]:
# Clean up local files (frames are already saved to Google Drive)
# Uncomment the lines below to clean up

# !rm -rf /content/downloads/*
# !rm -rf /content/videos/*
# print("✅ Local files cleaned up. Frames are safe in Google Drive!")

---

## 📋 Quick Reference

| Option | Description |
|--------|-------------|
| `-i` | Input directory with videos |
| `-u` | Text file with video URLs |
| `-o` | Output directory (use Drive path!) |
| `-c` | Parallel processing (default: auto) |
| `--fps` | Frames per second to extract |
| `--start` | Start time (seconds or HH:MM:SS) |
| `--end` | End time (seconds or HH:MM:SS) |
| `-q` | Quality 0-9 (0=best, 9=fastest) |
| `-v` | Verbose output |